In [ ]:
%pip install -r requirements.txt

In [ ]:
#GENERADOR DE ARCHIVOS CSV FINALES PARA CONSUMIR Y GENERAR EL .TTL

import csv
import pandas as pd

def getCoordenadas(csv_file):
    coordenadas = set()
    with open(csv_file, "r", encoding="utf-8") as file:

        lector = csv.DictReader(file)

        for fila in lector:
            
            lat = fila.get("decimalLatitude")
            lon = fila.get("decimalLongitude")
            especie = fila.get("scientificName")
            reino = fila.get("kingdom")
            nombre_comun = fila.get("vernacularName")

            if lat and lon:
                coordenadas.add((float(lat), float(lon), especie, reino, nombre_comun))
    return coordenadas

coordenadas_lp = getCoordenadas("csv/ocurrence_la_plata.csv")
coordenadas_bsas = getCoordenadas("csv/ocurrence_prov_bsas.csv")

coordenadas_union = coordenadas_lp.union(coordenadas_bsas) # ya esta

df_especies = pd.DataFrame(coordenadas_union, columns=["lat", "long", "especie", "reino", "nombre_comun"]) 

df_especies = pd.DataFrame(df_especies, columns=["lat", "long", "especie", "reino", "nombre_comun"])

coordenadas_personalizado = getCoordenadas("csv/ocurrence_personalizado.csv")

df_personalizado = pd.DataFrame(coordenadas_personalizado, columns=["lat", "long", "especie", "reino", "nombre_comun"])

#aca uno el dataframe de especies(coordenadas_union) y el dataframe personalizado
df_union = pd.concat([df_especies, df_personalizado], ignore_index=True).drop_duplicates()

df_union.to_csv("csv/datos_especies_final.csv", index=False)

La siguiente celda contiene el código para cargar el .ttl que tendra información de cada inmueble visto de la ontologia del OVS, con información acerca de 
especies vistas en un radio de 500 metros, también cuenta con la localización de cada observación individual.
Se puede omitir esta celda que tarda aprox. 28 minutos en ejecutarse, porque el .ttl ya se encuentra en la carpeta "grafos".

In [ ]:
#GENERADOR DEL .TTL, esta celda contiene el código para poder generar el archivo RDF
#tarda aprox. 28 minutos en generarlo

import pandas as pd
from rdflib import Graph, Literal, URIRef, Namespace
from rdflib.namespace import RDF, RDFS, XSD

#Def de prefijos
BIO_NS = Namespace("http://www.semanticweb.org/barre/ontologies/2025/8/bio-ontology/")
OVS_NS = Namespace("http://www.semanticweb.org/luciana/ontologies/2024/8/inmontology")
DWC_NS = Namespace("http://rs.tdwg.org/dwc/terms/") 

try: 
    df_total = pd.read_csv("datos_para_rdf/datos_con_conteo_total.csv")
    df_detalles = pd.read_csv("datos_para_rdf/datos_especies_detallados.csv")
    df_observaciones = pd.read_csv("csv/datos_especies_final.csv")
except FileNotFoundError as e:
    print(f"Error: {e}.")
    exit()

BUFFER_RADIUS_METERS = 500

g_biodiv_data = Graph()
g_biodiv_data.bind("bio", BIO_NS)
g_biodiv_data.bind("io", OVS_NS)
g_biodiv_data.bind("dwc", DWC_NS)
g_biodiv_data.bind("rdfs", RDFS)
g_biodiv_data.bind("xsd", XSD)


#agregado de inmuebles
print("Procesando inmuebles")
for index, row in df_total.iterrows():
    inmueble_uri_str = str(row['URI_Inmueble'])
    if pd.isna(inmueble_uri_str):
        continue
        
    inmueble_uri = URIRef(inmueble_uri_str)
    inmueble_uri_part = inmueble_uri_str.split('#')[-1]
    agg_uri = BIO_NS[f"Agg_{inmueble_uri_part}"]
    
    #triples del agregado
    g_biodiv_data.add((agg_uri, RDF.type, BIO_NS.ObservationAggregate))
    g_biodiv_data.add((agg_uri, BIO_NS.observedAt, inmueble_uri))
    g_biodiv_data.add((agg_uri, BIO_NS.totalObservations, Literal(row['Total_Especies_Unicas'], datatype=XSD.integer)))
    g_biodiv_data.add((agg_uri, BIO_NS.bufferRadius, Literal(BUFFER_RADIUS_METERS, datatype=XSD.integer)))
    
    wkt_point = f"POINT({row['Longitud']} {row['Latitud']})"
    g_biodiv_data.add((agg_uri, BIO_NS.asWKT, Literal(wkt_point, datatype=XSD.string)))
    
    #procesar los detalles de especies
    detalles_inmueble = df_detalles[df_detalles['URI_Inmueble'] == inmueble_uri_str]
    
    for _, detalle in detalles_inmueble.iterrows():
        especie_cientifica = detalle['especie']
        conteo_especie = detalle['conteo_especie_en_buffer']
        
        if pd.isna(especie_cientifica):
            continue

        especie_uri = BIO_NS[f"Taxon_{especie_cientifica.replace(' ', '_')}"]
        
        g_biodiv_data.add((especie_uri, RDF.type, BIO_NS.Taxon))
        
        if pd.notna(detalle['nombre_comun']):
            g_biodiv_data.add((especie_uri, RDFS.label, Literal(detalle['nombre_comun'], lang='es')))
        
        g_biodiv_data.add((agg_uri, BIO_NS.hasSpecies, especie_uri))
        g_biodiv_data.add((especie_uri, BIO_NS.speciesCountForAggregate, Literal(conteo_especie, datatype=XSD.integer)))

print(f"Procesados {len(df_total)} inmuebles")

#procesamieno de observaciones individuales
print("\nProcesando observaciones individuales")
observaciones_procesadas = 0

for index, row in df_observaciones.iterrows():
    lat = row['lat']
    lon = row['long']
    especie_cientifica = row['especie']
    reino = row['reino']
    nombre_comun = row['nombre_comun']
    
    if pd.isna(especie_cientifica) or pd.isna(lat) or pd.isna(lon):
        continue
    
    obs_uri = BIO_NS[f"Obs_{index}"]
    
    g_biodiv_data.add((obs_uri, RDF.type, BIO_NS.IndividualObservation))
    
    #agregar las coords.
    wkt_obs = f"POINT({lon} {lat})"
    g_biodiv_data.add((obs_uri, BIO_NS.asWKT, Literal(wkt_obs, datatype=XSD.string)))
    
    #vinculo con la especie
    especie_uri = BIO_NS[f"Taxon_{especie_cientifica.replace(' ', '_')}"]
    g_biodiv_data.add((obs_uri, BIO_NS.hasSpecies, especie_uri))
    
    g_biodiv_data.add((especie_uri, RDF.type, BIO_NS.Taxon))
    
    if pd.notna(reino) and str(reino).strip() != '':
        g_biodiv_data.add((obs_uri, BIO_NS.kingdom, Literal(str(reino), datatype=XSD.string)))
    
    if pd.notna(nombre_comun) and str(nombre_comun).strip() != '':
        g_biodiv_data.add((especie_uri, RDFS.label, Literal(str(nombre_comun), lang='es')))
    
    observaciones_procesadas += 1

print(f"Hay {observaciones_procesadas} observaciones individuales")

output_file = "grafos/biodiv_inmuebles_la_plata_COMPLETO.ttl"

with open(output_file, "w", encoding="utf-8") as f:
    f.write(g_biodiv_data.serialize(format="turtle"))

In [ ]:
#esta es la query basica para poder generar los mapas de calor, la query trae datos de cada inmueble, con sus coordenadas y
#la cantidad total de especies vistas en cada uno.
import pandas as pd

from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://localhost:3030/OVS-Biodiversidad/query")

sparql.setQuery("""
PREFIX bio: <http://www.semanticweb.org/barre/ontologies/2025/8/bio-ontology/>

SELECT ?InmuebleURI ?PuntoWKT ?ConteoTotal 
WHERE {
  ?Agregado bio:observedAt ?InmuebleURI .

  ?Agregado bio:asWKT ?PuntoWKT .

  ?Agregado bio:totalObservations ?ConteoTotal .
  
}
""")
sparql.setReturnFormat(JSON)

try:
    results = sparql.query().convert()

    data = []
    #results['results']['bindings'] contiene la lista de todas las filas devueltas
    for result in results['results']['bindings']:
        data.append({
            'URI_Inmueble': result['InmuebleURI']['value'],
            'WKT_Geometria': result['PuntoWKT']['value'],
            'Total_Especies_Unicas': float(result['ConteoTotal']['value'])
        })

    df_sparql = pd.DataFrame(data)
    
    print("-" * 40)
    print(f"Total de registros obtenidos: {len(df_sparql)}")
    display(df_sparql)
    print("-" * 40)
    
except Exception as e:
    print(f"Error al ejecutar la consulta SPARQL: {e}")

In [2]:
#para guardar el dataframe en un archivo CSV
df_sparql.to_csv("mapa_calor_sparql/datos_basicos.csv", index=False)

### Idea central

La idea de la combinación de los datasets fue poder mostrar el impacto que tienen los inmuebles sobre la biodiversidad en el partido de La Plata. 
A los datos obtenidos del OVS se les hizo un análisis geoespacial con la API de Google Maps para poder obtener las coordenadas (no venían incluidas en el OVS). Luego se pasaron esos datos a un CSV para facilitar la extracción y análisis de datos, y se combinaron esos datos con un archivo .CSV que contenía información de especies vistas en el partido de La Plata, este CSV lo extraje desde INaturalist. Para cada inmueble se le asigno un radio de 500 metros para el análisis de las especies cercanas a este y se guardo en un csv la información relacionando a cada inmueble con sus coordenadas y una lista con el nombre de cada especie vista. Después se paso toda esta información a un archivo .ttl, que tenía las bases del OVS, y se le sumo información relacionada a la biodiversidad, para poder hacer consultas en una base de datos no relacional como SPARQL. Para finalizar muestro las consultas SPARQL necesarias para obtener datos y poder generar los mapas de calor para consumir la información visualmente y poder tener un análisis más cómodo.
A continuación se generarán mapas con información relacionada a:
- Mapa que muestra para cada inmueble la cantidad de especies distintas que tienen en un radio de 500 metros. A cada inmueble se le asigna un color (rojo, verde, amarillo o gris) dependiendo de la cantidad de especies cercanas.
- Mapa de calor que muestra la distribución de las especies.
- Mapa de claor que muestra la distribución de los inmuebles.

In [ ]:
#este script genera el mapa de calor que define para cada inmueble un color
#dependiendo de la cantidad de especies únicas vistas en el mismo
import folium
import webbrowser
from shapely import wkt
import pandas as pd

df_sparql = pd.read_csv("mapa_calor_sparql/datos_basicos.csv")

df_sparql['geometry'] = df_sparql['WKT_Geometria'].apply(wkt.loads)

mapa_impacto_inmuebles = folium.Map(location=[-34.921, -57.954], zoom_start=12) #coordenadas de La Plata

def get_color(conteo):
    if conteo == 0:
        return 'grey' #biodiversidad nula
    elif conteo < 5:
        return 'green' #biodiversidad baja
    elif conteo < 10:
        return 'orange' #biodiversidad media
    else:
        return 'red' #biodiversidad alta
    
for index, row in df_sparql.iterrows():
    geometria = row['geometry']
    conteo = row['Total_Especies_Unicas']
    uri = row['URI_Inmueble']

    popup_text = f"""
    <b>Inmueble URI:</b> {uri}<br>
    <b>Especies únicas:</b> {int(conteo)}
    """

    folium.CircleMarker(
        location=(geometria.y, geometria.x),
        radius=5,
        color=get_color(conteo),
        fill_opacity=0.6,
        popup=folium.Popup(popup_text)
    ).add_to(mapa_impacto_inmuebles)

mapa_html = "mapa_calor_sparql/mapa_impacto_inmuebles.html"
mapa_impacto_inmuebles.save(mapa_html)
webbrowser.open(mapa_html)

In [ ]:
#esta celda tiene el script para generar una consulta SPARQL
#que obtiene info. solamente de los inmuebles, para posteriormente generar el mapa de calor
import pandas as pd

from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://localhost:3030/OVS-Biodiversidad/query")

sparql.setQuery("""
PREFIX bio: <http://www.semanticweb.org/barre/ontologies/2025/8/bio-ontology/>

SELECT ?InmuebleURI ?PuntoWKT 
WHERE {
  ?Agregado bio:observedAt ?InmuebleURI .

  ?Agregado bio:asWKT ?PuntoWKT .
}
""")
sparql.setReturnFormat(JSON)

try:
    results = sparql.query().convert()

    data = []
    for result in results['results']['bindings']:
        data.append({
            'URI_Inmueble': result['InmuebleURI']['value'],
            'WKT_Geometria': result['PuntoWKT']['value']
        })

    df_sparql_inmuebles = pd.DataFrame(data)
    
    print("-" * 40)
    print(f"Total de registros obtenidos: {len(df_sparql_inmuebles)}")
    display(df_sparql_inmuebles)
    print("-" * 40)
    
    df_sparql_inmuebles.to_csv("mapa_calor_sparql/datos_inmuebles.csv", index=False)

except Exception as e:
    print(f"Error al ejecutar la consulta SPARQL: {e}")

In [ ]:
#Mapas de calor, de inmuebles
import folium
import webbrowser
from folium.plugins import HeatMap
from shapely import wkt
import pandas as pd

df_sparql_inmuebles = pd.read_csv("mapa_calor_sparql/datos_inmuebles.csv")

df_sparql_inmuebles['geometry'] = df_sparql_inmuebles['WKT_Geometria'].apply(wkt.loads)
df_sparql_inmuebles['coords'] = df_sparql_inmuebles['geometry'].apply(lambda g: [g.y, g.x])

#mapa de calor de inmuebles
mapa_calor_inmuebles = folium.Map(location=[-34.921, -57.954], zoom_start=12)
HeatMap(df_sparql_inmuebles['coords'].tolist()).add_to(mapa_calor_inmuebles)

mapa_html_inmuebles = "mapa_calor_sparql/mapa_calor_inmuebles.html"

mapa_calor_inmuebles.save(mapa_html_inmuebles)
webbrowser.open(mapa_html_inmuebles)


In [ ]:
#esta celda contiene el script para generar un mapa de calor a partir de la información de la cantidad de especies vistas en distintos puntos geográficos
#permitiendo generar un mapa de calor para ver la distribución y densidad de observaciones de especies
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://localhost:3030/OVS-Biodiversidad/query")

sparql.setQuery("""
PREFIX bio: <http://www.semanticweb.org/barre/ontologies/2025/8/bio-ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?obsURI ?coordenadasWKT ?especieURI ?nombreEspecie ?reino
WHERE {
  ?obsURI a bio:IndividualObservation ;
          bio:asWKT ?coordenadasWKT ;
          bio:hasSpecies ?especieURI .
  OPTIONAL { ?especieURI rdfs:label ?nombreEspecie . }
  OPTIONAL { ?obsURI bio:kingdom ?reino . }
}
ORDER BY ?especieURI
""")
sparql.setReturnFormat(JSON)

try:
    results = sparql.query().convert()

    data = []
    for result in results['results']['bindings']:
        data.append({
            'Observacion_URI': result['obsURI']['value'],
            'WKT_Geometria': result['coordenadasWKT']['value'],
            'Especie_URI': result['especieURI']['value'],
            'Nombre_Comun': result.get('nombreEspecie', {}).get('value', 'N/A'),
            'Reino': result.get('reino', {}).get('value', 'N/A')
        })

    df_sparql_especies = pd.DataFrame(data)
    
    display(df_sparql_especies.head(10))
    
    df_sparql_especies.to_csv("mapa_calor_sparql/datos_especies_individuales.csv", index=False)

except Exception as e:
    print(f"Error al ejecutar la consulta SPARQL: {e}")

In [ ]:
# Mapa calor de observaciones de especies
import folium
import webbrowser
from folium.plugins import HeatMap
from shapely import wkt
import pandas as pd

df_sparql_especies = pd.read_csv("mapa_calor_sparql/datos_especies_individuales.csv")

df_sparql_especies['geometry'] = df_sparql_especies['WKT_Geometria'].apply(wkt.loads)
df_sparql_especies['coords'] = df_sparql_especies['geometry'].apply(lambda g: [g.y, g.x])

#Mapa calor de especies
mapa_calor_especies = folium.Map(location=[-34.921, -57.954], zoom_start=12)
HeatMap(df_sparql_especies['coords'].tolist()).add_to(mapa_calor_especies)

mapa_html_especies = "mapa_calor_sparql/mapa_calor_especies.html"

mapa_calor_especies.save(mapa_html_especies)
webbrowser.open(mapa_html_especies)